In [12]:
"""
Sudoku game structure.
"""

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a,b):
    return [s+t for s in a for t in b]

boxes = cross(rows,cols)
row_units = [cross(s,cols) for s in rows]
col_units = [cross(rows,s) for s in cols]
square_units = [cross(s,t) for s in ('ABC','DEF','GHI') for t in ('123','456','789')]
unitlist = square_units + col_units + row_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


In [47]:
"""
Functions used to solve sudoku.
"""

def grid_values(grid):
    """
    Return dictionary by assigning the value to boxes
    
    Attribute
    ---------
    grid: String
        Contains value of each sudoku box
    """
    assert len(grid) == 81,"Must be 81 character"
    grid = dict(zip(boxes,grid))
    for s in grid:
        if '.' in grid[s]:
            grid[s]='123456789'
    return grid

def eliminate(values):
    """
    Return the dictionary by eliminating values that is not required in boxes.
    
    Attributes
    ----------
    values: dict
        Contains the value of the boxes
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values 

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def only_choice(values):
    """
    Return the dictionary by keeping the unique value for the peer.
    
    Attributes
    ----------
    values: dict
        Contains the value of the boxes.
    """
    for units in unitlist:
        for unit in units:
            for i in values[unit]:
                if len(i) != 1:
                    for j in i:
                        flag=1
                        for k in unit:
                            if k!=unit and j in values[k]:
                                flag=0
                                break
                        if flag:
                            values[i]=j
                            break
    return values

def reduce_puzzle(values):
    """
    Iterate the eliminate and only_choice function till changes is found else return values
    
    Attributes
    ----------
    values: dict
        Contains the values of the boxes
    """
    status = True
    count2=1
    while status:
        count1=0
        values = eliminate(values)
        values = only_choice(values)
        for i in values:
            count1+=len(values[i])
        if count1 == count2:
            status=False
        count2=count1
    for i in values:
        if len(values[i]) == 0:
            return False
    return values

def search(values):
    """
    Iterate till all the boxes are not filled and then return values if solved else false.
    
    Attributes
    ----------
    values: dict
        Contains the value of each boxes.
    """
    values=reduce_puzzle(values)
    if values is False:
        return False
    if all(len(values[i]) == 1 for i in values):
        return values
    n,pos = min((len(values[i]),i) for i in values if len(values[i]) > 1)
    for i in values[pos]:
        new_values=values.copy()
        new_values[pos]=i
        status = search(new_values)
        if status:
            return status
        
def check(grid):
    """
    Return true if sudoku is solved else false
    
    Attributes
    ----------
    grid: dict
        Contains value of each boxes
    """
    for i in grid:
        for j in peers[i]:
            if(grid[j] == grid[i]):
                return False
    return True

def solve(grid):
    """
    Display sudoku if solved else error
    
    Attributes
    ----------
    grid: string
        Contains value of the each boxes
    """  
    grid = grid_values(grid)
    grid = search(grid)
    if check(grid):
        print(display(grid))
    else:
        print("Unable to solve")
        

In [50]:
grid1 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
grid2 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

In [52]:
solve(grid2)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 
None
